All code is from the following Kaggle notebook: https://www.kaggle.com/code/cocoyachi/safedriver-xgboost-musthave-ch08

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import save_npz
from bayes_opt import BayesianOptimization

In [2]:
data_path = '../input/'

train = pd.read_csv(data_path + 'train.csv.zip', index_col='id')
test = pd.read_csv(data_path + 'test.csv.zip', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv.zip', index_col='id')

In [3]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) 

all_features = all_data.columns

In [4]:
cat_features = [feature for feature in all_features if 'cat' in feature]

onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

In [5]:
all_data['num_missing'] = (all_data==-1).sum(axis=1)

In [6]:
remaining_features = [feature for feature in all_features
                      if ('cat' not in feature and 'calc' not in feature)]

remaining_features.append('num_missing')

In [7]:
ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True
for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str) + '_'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str) + '_'

In [8]:
all_data['mix_ind']

0          2_2_5_1_0_0_1_0_0_0_0_0_0_0_11_0_1_0_
1           1_1_7_0_0_0_0_1_0_0_0_0_0_0_3_0_0_1_
2          5_4_9_1_0_0_0_1_0_0_0_0_0_0_12_1_0_0_
3           0_1_2_0_0_1_0_0_0_0_0_0_0_0_8_1_0_0_
4           0_2_0_1_0_1_0_0_0_0_0_0_0_0_9_1_0_0_
                           ...                  
1488023     0_1_6_0_0_0_1_0_0_0_0_0_0_0_2_0_0_1_
1488024    5_3_5_1_0_0_0_1_0_0_0_0_0_0_11_1_0_0_
1488025     0_1_5_0_0_1_0_0_0_0_0_0_0_0_5_0_0_1_
1488026    6_1_5_1_0_0_0_0_1_0_0_0_0_0_13_1_0_0_
1488027    7_1_4_1_0_0_0_0_1_0_0_0_0_0_12_1_0_0_
Name: mix_ind, Length: 1488028, dtype: object

In [9]:
cat_count_features = []
for feature in cat_features+['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x:
                                                           val_counts_dict[x])
    cat_count_features.append(f'{feature}_count')

In [10]:
from scipy import sparse

drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin',
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

In [11]:
num_train = len(train) 


X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

In [12]:
X.shape

(595212, 217)

In [13]:
def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    
    n_samples = y_true.shape[0]
    L_mid = np.linspace(1 / n_samples, 1, n_samples)

    pred_order = y_true[y_pred.argsort()]
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    G_pred = np.sum(L_mid - L_pred)

    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order) / np.sum(true_order)
    G_true = np.sum(L_mid - L_true)
    
    return G_pred / G_true

In [14]:
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      test_size=0.2,
                                                      random_state=0)

bayes_dtrain = xgb.DMatrix(X_train, y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

In [17]:
param_bounds = {'max_depth': (4,8),
                'subsample': (0.6, 0.9),
                'colsample_bytree': (0.7, 1.0),
                'min_child_weight': (5, 7),
                'gamma': (8, 11),
                'reg_alpha': (7, 9),
                'reg_lambda': (1.1, 1.5),
                'scale_pos_weight': (1.4, 1.6)}


fixed_params = {'objective' : 'binary:logistic',
                'learning_rate': 0.02,
                'tree_method' : 'gpu_hist',
                'random_state': 1993}

In [18]:
def eval_function(max_depth, subsample, colsample_bytree, min_child_weight,
                 reg_alpha, gamma, reg_lambda, scale_pos_weight):

    params = {'max_depth': int(round(max_depth)),
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'min_child_weight': min_child_weight,
              'gamma': gamma,
              'reg_alpha':reg_alpha,
              'reg_lambda': reg_lambda,
              'scale_pos_weight': scale_pos_weight}

    params.update(fixed_params)
    
    print('hyperparameters :', params)    
        
    # XGBoost model training
    xgb_model = xgb.train(params=params, 
                          dtrain=bayes_dtrain,
                          num_boost_round=2000,
                          evals=[(bayes_dvalid, 'bayes_dvalid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=False)
                           
    best_iter = xgb_model.best_iteration # optimal number of iterations
    # Make predictions with validation data
    preds = xgb_model.predict(bayes_dvalid, 
                              iteration_range=(0, best_iter))
    # Gini coefficient calculation
    gini_score = eval_gini(y_valid, preds)
    print(f'Gini coefficient: {gini_score}\n')
    
    return gini_score


In [19]:
%%time
optimizer = BayesianOptimization(f=eval_function, 
                                 pbounds=param_bounds, 
                                 random_state=0)


optimizer.maximize(init_points=3, n_iter=6)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
hyperparameters : {'max_depth': 6, 'subsample': 0.867531900234624, 'colsample_bytree': 0.8646440511781974, 'min_child_weight': 6.0897663659937935, 'gamma': 10.14556809911726, 'reg_alpha': 7.84730959867781, 'reg_lambda': 1.3583576452266626, 'scale_pos_weight': 1.4875174422525386, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'tree_method': 'gpu_hist', 'random_state': 1993}


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/training.py:36: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  "`feval` is deprecated, use `custom_metric` instead.  They have "


Gini coefficient: 0.2770649772924368

| 1         | 0.2771    | 0.8646    | 10.15     | 6.411     | 6.09      | 7.847     | 1.358     | 1.488     | 0.8675    |
hyperparameters : {'max_depth': 7, 'subsample': 0.6261387899104622, 'colsample_bytree': 0.9890988281503088, 'min_child_weight': 6.0577898395058085, 'gamma': 9.150324556477333, 'reg_alpha': 8.136089122187865, 'reg_lambda': 1.4702386553170643, 'scale_pos_weight': 1.4142072116395774, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'tree_method': 'gpu_hist', 'random_state': 1993}
Gini coefficient: 0.2795461824105286

| 2         | 0.2795    | 0.9891    | 9.15      | 7.167     | 6.058     | 8.136     | 1.47      | 1.414     | 0.6261    |
hyperparameters : {'max_depth': 7, 'subsample': 0.8341587528859367, 'colsample_bytree': 0.7060655192320977, 'min_child_weight': 6.7400242964936385, 'gamma': 10.497859536643814, 'reg_alpha': 8.957236684465528, 'reg_lambda': 1.4196634256866894, 'scale_pos_weight': 1.4922958724505864, 'objective'

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/training.py:36: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  "`feval` is deprecated, use `custom_metric` instead.  They have "


Gini coefficient: 0.277282372969139

| 4         | 0.2773    | 0.8843    | 10.45     | 6.838     | 6.494     | 8.552     | 1.381     | 1.423     | 0.7002    |
hyperparameters : {'max_depth': 7, 'subsample': 0.6, 'colsample_bytree': 1.0, 'min_child_weight': 5.952078321630083, 'gamma': 8.64637619160198, 'reg_alpha': 8.145017582466787, 'reg_lambda': 1.5, 'scale_pos_weight': 1.4, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'tree_method': 'gpu_hist', 'random_state': 1993}


...

Gini coefficient: 0.2814854045751619

| 7         | 0.2815    | 1.0       | 8.0       | 8.0       | 5.0       | 7.0       | 1.1       | 1.4       | 0.6       |
hyperparameters : {'max_depth': 6, 'subsample': 0.6, 'colsample_bytree': 1.0, 'min_child_weight': 7.0, 'gamma': 8.0, 'reg_alpha': 7.0, 'reg_lambda': 1.1, 'scale_pos_weight': 1.4, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'tree_method': 'gpu_hist', 'random_state': 1993}


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/training.py:36: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  "`feval` is deprecated, use `custom_metric` instead.  They have "


Gini coefficient: 0.2820560768191399

| 8         | 0.2821    | 1.0       | 8.0       | 6.246     | 7.0       | 7.0       | 1.1       | 1.4       | 0.6       |
hyperparameters : {'max_depth': 4, 'subsample': 0.6, 'colsample_bytree': 1.0, 'min_child_weight': 7.0, 'gamma': 8.0, 'reg_alpha': 7.0, 'reg_lambda': 1.1, 'scale_pos_weight': 1.4, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'tree_method': 'gpu_hist', 'random_state': 1993}


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/training.py:36: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  "`feval` is deprecated, use `custom_metric` instead.  They have "


Gini coefficient: 0.28171588531798414

| 9         | 0.2817    | 1.0       | 8.0       | 4.0       | 7.0       | 7.0       | 1.1       | 1.4       | 0.6       |
CPU times: user 6min 20s, sys: 20 s, total: 6min 40s
Wall time: 6min 7s


In [20]:
max_params = optimizer.max['params']
max_params

{'colsample_bytree': 1.0,
 'gamma': 8.0,
 'max_depth': 6.245597321042113,
 'min_child_weight': 7.0,
 'reg_alpha': 7.0,
 'reg_lambda': 1.1,
 'scale_pos_weight': 1.4,
 'subsample': 0.6}

In [21]:
max_params['max_depth'] = int(round(max_params['max_depth']))

max_params.update(fixed_params)
max_params

{'colsample_bytree': 1.0,
 'gamma': 8.0,
 'max_depth': 6,
 'min_child_weight': 7.0,
 'reg_alpha': 7.0,
 'reg_lambda': 1.1,
 'scale_pos_weight': 1.4,
 'subsample': 0.6,
 'objective': 'binary:logistic',
 'learning_rate': 0.02,
 'tree_method': 'gpu_hist',
 'random_state': 1993}

In [22]:
%%time

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)


oof_val_preds = np.zeros(X.shape[0]) 
oof_test_preds = np.zeros(X_test.shape[0]) 
dtest = xgb.DMatrix(X_test)

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):

    print('#'*40, f'Fold {idx+1} / {folds.n_splits}', '#'*40)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]


    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
   

    xgb_model = xgb.train(params=max_params, 
                          dtrain=dtrain,
                          num_boost_round=2000,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=100)

    best_iter = xgb_model.best_iteration
    oof_test_preds += xgb_model.predict(dtest,
                                        iteration_range=(0, best_iter))/folds.n_splits
    
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid, 
                                                  iteration_range=(0, best_iter))
    
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'Fold {idx+1} gini score : {gini_score}\n')

######################################## Fold 1 / 5 ########################################
[0]	valid-logloss:0.67660	valid-gini:0.16913


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/training.py:36: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  "`feval` is deprecated, use `custom_metric` instead.  They have "


[100]	valid-logloss:0.19038	valid-gini:0.24909
[200]	valid-logloss:0.15745	valid-gini:0.27772
[300]	valid-logloss:0.15436	valid-gini:0.28687
[400]	valid-logloss:0.15389	valid-gini:0.29010
[500]	valid-logloss:0.15380	valid-gini:0.29142
[600]	valid-logloss:0.15376	valid-gini:0.29230
[700]	valid-logloss:0.15373	valid-gini:0.29288
[800]	valid-logloss:0.15372	valid-gini:0.29350
[900]	valid-logloss:0.15372	valid-gini:0.29351
[1000]	valid-logloss:0.15370	valid-gini:0.29351
[1100]	valid-logloss:0.15371	valid-gini:0.29357
[1200]	valid-logloss:0.15371	valid-gini:0.29347
[1239]	valid-logloss:0.15372	valid-gini:0.29353
Fold 1 gini score : 0.29358331406338356

######################################## Fold 2 / 5 ########################################
[0]	valid-logloss:0.67661	valid-gini:0.12431
[100]	valid-logloss:0.19046	valid-gini:0.23843
[200]	valid-logloss:0.15769	valid-gini:0.26435
[300]	valid-logloss:0.15469	valid-gini:0.27355
[400]	valid-logloss:0.15426	valid-gini:0.27679
[500]	valid-loglos

In [23]:
print('OOF Verification data Gini coefficient:', eval_gini(y, oof_val_preds))


OOF Verification data Gini coefficient: 0.28588719996388795


In [24]:
submission['target'] = oof_test_preds
submission.to_csv('../submissions/submission_best_kaggle_xgb_gpu_hist.csv')

0.28457 Public, 0.28802 private

In [33]:
X.shape

(595212, 217)

In [36]:
X_test.shape

(892816, 217)

In [37]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [38]:
X

<595212x217 sparse matrix of type '<class 'numpy.float64'>'
	with 24449411 stored elements in Compressed Sparse Row format>

In [39]:
X.toarray()

array([[ 2.,  5.,  0., ...,  0.,  0.,  0.],
       [ 1.,  7.,  0., ...,  0.,  0.,  0.],
       [ 5.,  9.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 1., 10.,  1., ...,  0.,  0.,  0.],
       [ 5.,  3.,  0., ...,  0.,  0.,  0.],
       [ 0.,  8.,  1., ...,  0.,  0.,  0.]])

In [41]:
save_npz('../input/X_217.npz', X)

In [42]:
save_npz('../input/X_test_217.npz', X_test)

In [43]:
np.save('../input/y_217.npy', y)